In [1]:
import pandas as pd

In [ ]:
# dataset origin: https://www.kaggle.com/datasets/pierremegret/dialogue-lines-of-the-simpsons
df = pd.read_csv('../data/raw/simpsons_dataset.csv')
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [3]:
MINIMUM_LINE_LENGTH = 15
df = df[df['spoken_words'].str.split().str.len() >= MINIMUM_LINE_LENGTH]                         # remove lines with less than 15 words
df.groupby('raw_character_text').size().sort_values(ascending=False).head(10)                    # count number of lines per character

raw_character_text
Homer Simpson          5779
Marge Simpson          2561
Lisa Simpson           2026
Bart Simpson           1938
C. Montgomery Burns     894
Moe Szyslak             785
Seymour Skinner         724
Ned Flanders            547
Krusty the Clown        504
Kent Brockman           474
dtype: int64

In [4]:
df['words'] = df['spoken_words'].str.split().str.len()

In [ ]:
minor_characters_list = ['C. Montgomery Burns', 'Seymour Skinner', 'Ned Flanders']
minor_characters = {}
for char in minor_characters_list:
    minor_characters[char] = df[df['raw_character_text'] == char].sort_values(by='words', ascending=False).iloc[:300].reset_index(drop=True)

In [ ]:
# save full and sampled datasets for each character
N_SAMPLES = 100
sample_bychar_df = {}

for name in minor_characters_list:
    dataset = minor_characters[name].drop(columns='words')

    name = name.lower().replace(' ', '_').replace('.', '')
    dataset.to_csv(f'../data/raw/raw-char-lines/{name}_lines.csv', index=False)

    sample_bychar_df[name] = dataset.drop(columns='raw_character_text')
    sample_bychar_df[name] = sample_bychar_df[name].sample(n=N_SAMPLES)

    sample_bychar_df[name].to_csv(f'../data/raw/sampled-char-lines/{name}_lines_sampled.csv', index=False)

In [ ]:
# create master sampled dataframe with shuffled samples from all selected minor characters
sample_master_df = pd.concat(sample_bychar_df.values()).sample(frac=1, random_state=42).reset_index(drop=True)
sample_master_df.to_csv('../data/raw/sampled-char-lines/all_minor_characters_sampled.csv', index=False)